In [1]:
import datasets
from datasets import Dataset, DatasetDict
import pandas as pd
import transformers

/Users/apsys/Downloads/startup-imperial/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tdf=pd.read_csv('train.csv',dtype={'text':str,'labels':int}).rename(columns={'labels':'label'}).reset_index(drop=True)
vdf=pd.read_csv('test.csv',dtype={'text':str,'labels':int}).rename(columns={'labels':'label'}).reset_index(drop=True)

tds = Dataset.from_pandas(tdf)
vds = Dataset.from_pandas(vdf)


ds = DatasetDict()

ds['train'] = tds
ds['validation'] = vds


# labels = [label for label in ds['train'].features.keys() if label not in ['Unnamed: 0','text']]
# id2label = {idx:label for idx, label in enumerate(labels)}
# label2id = {label:idx for idx, label in enumerate(labels)}
# labels

In [21]:
tds

Dataset({
    features: ['Unnamed: 0', 'text', 'label'],
    num_rows: 465
})

In [4]:
from transformers import AutoTokenizer
import numpy as np
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = ds.map(tokenize_function, batched=True)

100%|██████████| 1/1 [00:00<00:00, 189.64ba/s]


In [23]:
tokenized_datasets['train']

Dataset({
    features: ['Unnamed: 0', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 465
})

In [5]:
from transformers import AutoModelForSequenceClassification, BertForSequenceClassification
from transformers import TrainingArguments,Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")
model = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=10)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [6]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics,
)

In [9]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, Unnamed: 0. If text, Unnamed: 0 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/Users/apsys/Downloads/startup-imperial/.venv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 465
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 177
  Number of trainable parameters = 108317962
 33%|███▎      | 59/177 [13:05<19:54, 10.12s/it]The following columns in the evaluat

{'eval_loss': 2.1510345935821533, 'eval_accuracy': 0.21428571428571427, 'eval_runtime': 5.0065, 'eval_samples_per_second': 2.796, 'eval_steps_per_second': 0.399, 'epoch': 1.0}


 67%|██████▋   | 118/177 [27:07<10:11, 10.36s/it]The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, Unnamed: 0. If text, Unnamed: 0 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 14
  Batch size = 8
                                                 
 67%|██████▋   | 118/177 [27:12<10:11, 10.36s/it]

{'eval_loss': 2.055898427963257, 'eval_accuracy': 0.21428571428571427, 'eval_runtime': 5.2442, 'eval_samples_per_second': 2.67, 'eval_steps_per_second': 0.381, 'epoch': 2.0}


100%|██████████| 177/177 [1:18:56<00:00,  8.88s/it]   The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, Unnamed: 0. If text, Unnamed: 0 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 14
  Batch size = 8
                                                   
100%|██████████| 177/177 [1:19:00<00:00,  8.88s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 177/177 [1:19:00<00:00, 26.78s/it]

{'eval_loss': 2.0245931148529053, 'eval_accuracy': 0.21428571428571427, 'eval_runtime': 3.7326, 'eval_samples_per_second': 3.751, 'eval_steps_per_second': 0.536, 'epoch': 3.0}
{'train_runtime': 4740.1837, 'train_samples_per_second': 0.294, 'train_steps_per_second': 0.037, 'train_loss': 2.2401129943502824, 'epoch': 3.0}


TrainOutput(global_step=177, training_loss=2.2401129943502824, metrics={'train_runtime': 4740.1837, 'train_samples_per_second': 0.294, 'train_steps_per_second': 0.037, 'train_loss': 2.2401129943502824, 'epoch': 3.0})

In [10]:
from transformers.onnx import FeaturesManager
from pathlib import Path

feature = "sequence-classification"
model_kind, model_onnx_config = FeaturesManager.check_supported_model_or_raise(model, feature=feature)
onnx_config = model_onnx_config(model.config)

# export
onnx_inputs, onnx_outputs = transformers.onnx.export(
        preprocessor=tokenizer,
        model=model,
        config=onnx_config,
        opset=13,
        output=Path("trfs-model.onnx")
)

Using framework PyTorch: 1.13.1
Overriding 1 configuration item(s)
	- use_cache -> False


In [11]:
from transformers import TextClassificationPipeline

model = model
tokenizer = tokenizer
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, top_k=None)

In [13]:
pipe("Coupons & Cashback App for online shopping We are Growing $300K MONTHLY GMV 37%")

[[{'label': 'LABEL_4', 'score': 0.22812390327453613},
  {'label': 'LABEL_0', 'score': 0.13694903254508972},
  {'label': 'LABEL_3', 'score': 0.1114092543721199},
  {'label': 'LABEL_6', 'score': 0.10610835999250412},
  {'label': 'LABEL_1', 'score': 0.09675037860870361},
  {'label': 'LABEL_2', 'score': 0.08507678657770157},
  {'label': 'LABEL_7', 'score': 0.08245889097452164},
  {'label': 'LABEL_8', 'score': 0.06800109148025513},
  {'label': 'LABEL_9', 'score': 0.055534619837999344},
  {'label': 'LABEL_5', 'score': 0.029587682336568832}]]